In [96]:
import torch
from botorch.models import SingleTaskGP
from botorch.models.transforms import Normalize, Standardize
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition import LogExpectedImprovement, UpperConfidenceBound, ProbabilityOfImprovement
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.optim import optimize_acqf
import numpy as np
import matplotlib.pyplot as plt

def poly(x):
    return (-0.001 * x**6 + 0.003 * x**5 + 0.062 * x**4 - 
            0.174 * x**3 - 0.673 * x**2 + 1.323 * x + 1.764)



In [ ]:
def plot_acq_func(train_x, train_y, acq_func, model, true_y = None):

    # Plot preperations
    plot_x = np.linspace(-5,5,100)
    plot_x_tensor = torch.tensor(plot_x.reshape(-1,1))
    plot_x_tensor = torch.tensor(plot_x.reshape(-1,1))

    # Find the mean
    with torch.no_grad():
        model_mean = model.posterior(plot_x_tensor).mean, model.posterior(plot_x_tensor).variance

    acq_dict={
        'LogEI': LogExpectedImprovement(model=model, best_f=train_y.max()),
        'UCB': ProbabilityOfImprovement(model=model, best_f=train_y.max()),
        'LogPI':ProbabilityOfImprovement(model=model, best_f=train_y.max())
    }
    
    acq_trained = acq_dict[acq_func](model=model, best_f=train_y.max())

    bounds = torch.stack([torch.zeros(1), torch.ones(1)]).to(torch.double)
    candidate = optimize_acqf(acq_trained, bounds=bounds, q=1, num_restarts=5, raw_samples=20)

    fig, axs = plt.subplots(2)
    fig.set_size_inches(7, 7)
    #axs[0].plot(plot_x,plot_y)
    axs[0].scatter(train_x,train_y, color = 'orange')
    axs[0].plot(plot_x_tensor.numpy(), model_mean.numpy(), label="GP Mean", color="red", linestyle='dashed')
    axs[0].scatter(candidate, poly(candidate), color = 'green', marker='o', s= 50)

    #ax2.set_ylim(-20,10)
    axs[1].plot(plot_x_tensor.numpy(), acq_trained.numpy(), color="orange", linestyle='dashed')

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from botorch.models import SingleTaskGP
from botorch.acquisition.multi_objective.monte_carlo import qExpectedHypervolumeImprovement
from botorch.optim import optimize_acqf
from botorch.models.transforms import Normalize, Standardize
from botorch.utils.multi_objective.box_decompositions.non_dominated import NondominatedPartitioning
from gpytorch.likelihoods import GaussianLikelihood

# Step 1: Generate 2D synthetic training data
torch.manual_seed(42)  # For reproducibility
train_X = torch.rand(10, 2) * 10 - 5  # 10 points in [-5,5] x [-5,5]
train_Y1 = torch.sin(train_X[:, 0]) + torch.cos(train_X[:, 1]) + 0.1 * torch.randn(10)
train_Y2 = torch.cos(train_X[:, 0]) - torch.sin(train_X[:, 1]) + 0.1 * torch.randn(10)
train_Y = torch.stack([train_Y1, train_Y2], dim=-1)  # Multi-objective outputs

# Step 2: Define GP Model
gp = SingleTaskGP(
    train_X,
    train_Y,
    input_transform=Normalize(d=2),
    outcome_transform=Standardize(m=2),
)
gp.eval()

# Step 3: Define Reference Point (for Hypervolume Calculation)
ref_point = train_Y.min(dim=0).values - 0.1  # Slightly below the minimum observed

# Step 4: Define EHVI Acquisition Function
partitioning = NondominatedPartitioning(ref_point=ref_point, Y=train_Y)
ehvi = qExpectedHypervolumeImprovement(model=gp, ref_point=ref_point.tolist(), partitioning=partitioning)

# Step 5: Generate a 2D grid for plotting
X1 = torch.linspace(-5, 5, 30)
X2 = torch.linspace(-5, 5, 30)
X1_mesh, X2_mesh = torch.meshgrid(X1, X2, indexing="ij")
X_grid = torch.cat([X1_mesh.reshape(-1, 1), X2_mesh.reshape(-1, 1)], dim=1)

# Step 6: Compute EHVI over the grid
with torch.no_grad():
    EHVI_values = ehvi(X_grid).reshape(30, 30)  # Reshape for 3D plotting

# Step 7: 3D Plot of EHVI
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(X1_mesh.numpy(), X2_mesh.numpy(), EHVI_values.numpy(), cmap="viridis", alpha=0.8)
ax.set_xlabel("X1")
ax.set_ylabel("X2")
ax.set_zlabel("EHVI")
ax.set_title("Expected Hypervolume Improvement (EHVI)")

plt.show()


C:\Users\pc\AppData\Local\Temp\ipykernel_31704\1202284431.py:20: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  gp = SingleTaskGP(
c:\Users\pc\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\acquisition\multi_objective\monte_carlo.py:111: NumericsWarning: qExpectedHypervolumeImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 qExpectedHypervolumeImprovement 	 --> 	 qLogExpectedHypervolumeImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)
c:\Users\pc\AppData\Local\Programs\Python\Python313\Lib\site-packages\linear_operator\utils\cholesky.py:40:

In [109]:
# Generate training data
train_x = np.random.uniform(low=-5, high=5, size=10).astype(np.float32)  # Convert to float32
train_y = poly(train_x).astype(np.float32)  # Convert to float32

# Convert to PyTorch tensors with float32 dtype
train_x = torch.tensor(train_x.reshape(-1,1), dtype=torch.float32)
train_y = torch.tensor(train_y.reshape(-1,1), dtype=torch.float32)

# Define GP model
gp = SingleTaskGP(
    train_X=train_x,
    train_Y=train_y,
    input_transform=Normalize(d=1),
    outcome_transform=Standardize(m=1),
)

# Generate test points
plot_x = np.linspace(-5, 5, 100).astype(np.float32)  # Convert to float32
plot_y = poly(plot_x).astype(np.float32)  # Convert to float32
plot_x_tensor = torch.tensor(plot_x.reshape(-1,1), dtype=torch.float32)  # Ensure float32

# Compute posterior mean and variance
with torch.no_grad():
    gp_mean = gp.posterior(plot_x_tensor).mean  # Now both are float32
    gp_var = gp.posterior(plot_x_tensor).variance
    
plot_acq_func(train_x, train_y, 'LogEI', gp)

C:\Users\pc\AppData\Local\Temp\ipykernel_31704\1025535847.py:10: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  gp = SingleTaskGP(


RuntimeError: expected m1 and m2 to have the same dtype, but got: double != float

In [91]:
train_X = np.random.uniform(low = -5, high = 5, size = 10)
Y = poly(train_X)
Y = torch.tensor(Y.reshape(-1,1))# + 0.1 * torch.randn_like(Y)  # add some noise
train_X = torch.tensor(train_X.reshape(-1,1))

gp = SingleTaskGP(
  train_X=train_X,
  train_Y=Y,
  input_transform=Normalize(d=1),
  outcome_transform=Standardize(m=1)
)

plot_x = np.linspace(-5,5,100)
plot_y = poly(plot_x)
plot_x_tensor = torch.tensor(plot_x.reshape(-1,1))
plot_x_tensor = torch.tensor(plot_x.reshape(-1,1))

with torch.no_grad():
    gp_mean, gp_var = gp.posterior(plot_x_tensor).mean, gp.posterior(plot_x_tensor).variance

In [92]:
logEI = LogExpectedImprovement(model=gp, best_f=Y.max())
EI = logEI(plot_x_tensor.unsqueeze(-2)).detach()

bounds = torch.stack([torch.zeros(1), torch.ones(1)]).to(torch.double)
candidate, acq_value = optimize_acqf(
  logEI, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
)
num = candidate.item()
poly(num)

1.764